In [1]:
# GPU BEGONE!
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'
os.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ

environ{'USER': 'fandav',
        'JUPYTERHUB_HOST': '',
        'LD_LIBRARY_PATH': '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu',
        'JUPYTERHUB_USER': 'fandav',
        'SHLVL': '0',
        'HOME': '/home/fandav',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/fandav/oauth_callback',
        'SLURM': '1',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-fandav',
        'PATH': '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin',
        'JUPYTERHUB_ACTIVITY_URL': 'http://127.0.0.1:8081/hub/api/users/fandav/activity',
        'LANG': 'en_US.UTF-8',
        'SHELL': '/bin/zsh',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/fandav/',
        'PWD': '/home/fandav',
        'JUPYTERHUB_API_TOKEN': 'aa97b83622c84e4784a2118a82bf8903',
        'JUPYTERHUB_BASE_URL': '/',
        'XDG_DATA_DIRS': '/usr/local/share:/usr/share:/v

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

# To find out which devices your operations and tensors are assigned to
#tf.debugging.set_log_device_placement(True)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# Obtain Embeddings

In [3]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

# model
model = TFBertModel.from_pretrained('bert-base-uncased');
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


# Import Data

In [4]:
dataset = pd.read_csv('final_dataset.csv', index_col=0)
#display(dataset.loc['Content'].values)

In [5]:
# grab content
dataset_content = dataset.loc['Content'].fillna('').values
#print(dataset_content)

# process content through BERT
encoded_strings = [tokenizer.encode(content, max_length=512, pad_to_max_length=True) for content in dataset_content]
encoded_vectors = tf.constant(encoded_strings)
outputs = model(encoded_vectors)

# obtain per line embeddings from all the artibles from BERT
last_hidden_states = outputs[0]

# obtain article embeddings from BERT
article_embeddings = outputs[1]

print(np.shape(outputs))
print(np.shape(outputs[0]))
print(np.shape(outputs[1]))

(2, 50)
(50, 512, 768)
(50, 768)


In [6]:
_, num_articles = np.shape(outputs)
article_names = [f'Article {idx}' for idx in range(num_articles)]

# vector embeddings of each article
vec_embeddings = article_embeddings.numpy()

In [7]:
truth_labels = np.array(list(map(int, dataset.loc['Originality Score'].values)))
#truth_labels = np.array(list(map(int, dataset.loc['Fact-Checked'].values)))
#truth_labels = np.array(list(map(int, dataset.loc['Credibility Score'].values)))
print(f'Num truth_labels = {np.shape(truth_labels)}')
print(truth_labels)
print(f'Unique Values: {set(truth_labels)} | Num Unique Values: {len(set(truth_labels))}')

Num truth_labels = (50,)
[0 2 0 1 3 1 0 2 0 2 2 3 0 1 2 0 0 2 1 1 0 1 1 3 0 0 2 3 0 1 1 2 0 0 0 0 1
 0 0 1 2 0 0 0 0 0 0 0 0 0]
Unique Values: {0, 1, 2, 3} | Num Unique Values: 4


In [8]:
display(dataset)

,Article 43,Article 13,Article 14,Article 30,Article 23,Article 42,Article 49,Article 36,Article 45,Article 7,...,Article 44,Article 46,Article 35,Article 38,Article 37,Article 28,Article 29,Article 39,Article 41,Article 12
Title,Which Banana Would You Eat? Your Answer May Ha...,NOW IT’S OFFICIAL: FDA Announced That Vaccines...,CONFIRMED: E-CIGARETTES CAUSE A HORRIBLE INCUR...,WITH ONLY 2 CUPS A DAY FOR 1 WEEK YOUR STOMACH...,"Lead Developer of HPV Vaccines Comes Clean, Wa...",DELINGPOLE: Global Warming Study Cancelled Bec...,Diet drinks TRIPLE your risk of stroke and dem...,NaN,NaN,Nations Press: PLS SHARE: DO NOT EAT THIS FISH...,...,NaN,Opinion | How the Anti-Vaxxers Are Winning,Arctic’s Winter Sea Ice Drops to Its Lowest Re...,The Best Exercise for Aging Muscles,An Iceberg the Size of Delaware Just Broke Awa...,Putting Kids To Bed Early Improves Mom's Healt...,If Everyone Ate Beans Instead of Beef,Arctic stronghold of world’s seeds flooded aft...,"The oldest child is actually the smartest, stu...",Coconut oil isn't healthy. It's never been hea...
Content,"When we eat bananas, we do so because we are h...",The FDA has published conclusive proof on thei...,CONFIRMED: E-CIGARETTES CAUSE A HORRIBLE INCUR...,The desire of lots of people is to have a flat...,"Lead Developer of HPV Vaccines Comes Clean, Wa...",A global warming research study in Canada has ...,"The Boston University study of almost 4,400 ad...",NaN,NaN,NaN,...,NaN,A major measles outbreak in America is only a ...,Much of the ice also appears to be thinner tha...,Certain kinds of exercise may mitigate the eff...,A crack more than 120 miles long had developed...,A new study confirms why all those bedtime bat...,"With one dietary change, the U.S. could hypoth...",No seeds were lost but the ability of the rock...,"Birth order doesn't affect personality, but it...","“We advise against the use of coconut oil,"" th..."
URL,http://goodfullness.net/which-banana-would-you...,http://inshapetoday.com/now-official-fda-annou...,http://nowcheckthis.com/2017/03/23/confirmed-e...,http://publichealthabc.com/2-cups-day-1-week-s...,http://www.alternativenewsnetwork.net/lead-dev...,http://www.breitbart.com/big-government/2017/0...,http://www.dailymail.co.uk/~/article-4429790/i...,http://www.iflscience.com/environment/heavy-mo...,http://www.iflscience.com/health-and-medicine/...,http://www.nationspressph.com/2017/02/pls-shar...,...,https://www.ntd.tv/inspiring/parenting/sam-ber...,https://www.nytimes.com/2017/02/08/opinion/how...,https://www.nytimes.com/2017/03/22/climate/arc...,https://www.nytimes.com/2017/03/23/well/move/t...,https://www.nytimes.com/interactive/2017/06/09...,https://www.simplemost.com/new-study-says-putt...,https://www.theatlantic.com/health/archive/201...,https://www.theguardian.com/environment/2017/m...,https://www.today.com/health/birth-order-first...,https://www.usatoday.com/story/news/nation-now...
Credibility Score,1,0,1,0,0,1,1,0,1,0,...,2,4,4,3,4,2,4,4,3,3
Originality Score,0,2,0,1,3,1,0,2,0,2,...,2,0,0,0,0,0,0,0,0,0
Attribution Rank,1,2,0,1,3,1,0,2,0,2,...,3,0,0,0,0,0,0,0,0,0
Source 1 Category,0,2,2,0,1,3,2,2,2,0,...,3,2,2,2,3,2,3,3,2,2
Source 2 Category,0,1,0,0,1,3,3,2,3,0,...,3,3,2,1,2,2,2,2,2,2
Source 3 Category,0,1,1,0,1,3,1,2,3,0,...,3,1,0,0,1,2,2,2,1,3
Article has Spammy/Clickbaity Ads,2,3,1,3,2,4,4,3,4,4,...,3,1,0,0,0,4,1,1,3,2


# Define Neural Network Model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalMaxPooling1D, Conv1D, MaxPooling1D, InputLayer, Input, Softmax

In [10]:
def model_def():
    model = Sequential([
        Input(shape=(768, 1), name='input'),
        
        Conv1D(400, 3, padding='valid', activation='relu', strides=3, input_shape=(768,)),
        Dense(200, activation='relu', name='dense_0'),
        Dropout(0.2),
        
        Conv1D(200, 3, padding='valid', activation='relu', strides=3),
        Dense(150, activation='relu', name='dense_1'),
        Dropout(0.2),
        
        Conv1D(100, 3, padding='valid', activation='relu', strides=5),
        GlobalMaxPooling1D(),
        
        Dense(64, activation='relu', name='dense_2'),
        
        Dense(len(set(truth_labels)), name='dense_3', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Softmax()
    ]);
    #model.summary()

    model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
                  # Loss function to minimize
                  #loss=tf.keras.losses.SquaredHinge(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # List of metrics to monitor
                  metrics=['accuracy']);
    return model

In [11]:
# sample test run
n_epochs = 300

train_index = np.arange(1, 50)
test_index = np.arange(1)

X_train = vec_embeddings[train_index]
Y_train = truth_labels[train_index]

X_test = vec_embeddings[test_index]
Y_test = truth_labels[test_index]

X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)
print(np.shape(X_train))

# reset model
model = model_def()
model.summary()

# apply training set
history = model.fit(X_train, Y_train,
                    batch_size=10,
                    epochs=n_epochs, nb_epoch=10, verbose=1);
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(x_val, y_val))

(49, 768, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 256, 400)          1600      
_________________________________________________________________
dense_0 (Dense)              (None, 256, 200)          80200     
_________________________________________________________________
dropout_37 (Dropout)         (None, 256, 200)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 85, 200)           120200    
_________________________________________________________________
dense_1 (Dense)              (None, 85, 150)           30150     
_________________________________________________________________
dropout_38 (Dropout)         (None, 85, 150)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 17, 100

In [12]:
# apply testing set
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=1)
print(test_acc)

# predict a few samples
output = model.predict(X_train[0:20])
print(f'Output Vector: \n{output}')
print(f'Output Vector Rounded: \n{np.round(output)}')
print('Output Label: {}'.format([np.where(r==1)[0][0] for r in np.round(output)]))
print('Output Label: {}'.format(Y_train[0:20]))

1/1 [==============================] - 0s 888us/step - loss: 0.7443 - accuracy: 1.0000
1.0
Output Vector: 
[[1.00000000e+00 6.95951574e-10 1.00518179e-31 3.56947943e-21]
 [1.00000000e+00 6.94413138e-11 6.91963683e-32 1.17448338e-21]
 [1.00000000e+00 1.08315601e-09 2.16443446e-31 6.27132636e-21]
 [1.00000000e+00 1.29462774e-10 5.46554280e-30 1.40764265e-20]
 [1.00000000e+00 3.26157781e-11 1.12103517e-33 1.02354273e-22]
 [1.00000000e+00 2.67951009e-11 1.26547229e-33 1.03826218e-22]
 [8.37124864e-15 4.03439481e-05 9.99959469e-01 2.16616129e-07]
 [8.37124864e-15 4.03439481e-05 9.99959469e-01 2.16616129e-07]
 [8.37124864e-15 4.03439481e-05 9.99959469e-01 2.16616129e-07]
 [1.00000000e+00 1.90992755e-09 7.09823750e-30 4.88400456e-20]
 [1.00000000e+00 8.65674865e-10 5.38715096e-31 9.44286204e-21]
 [1.00000000e+00 2.75794144e-11 2.50960605e-34 4.25859807e-23]
 [1.00000000e+00 5.03362560e-11 8.09780573e-33 3.56437498e-22]
 [1.00000000e+00 1.82572471e-10 2.22347489e-32 9.92608657e-22]
 [1.0000000

In [31]:
from sklearn.model_selection import KFold

n_folds = 50
n_epochs = 1500
test_accuracy = []

kf = KFold(n_splits=n_folds)
kf.get_n_splits(vec_embeddings)
for k_index, (train_index, test_index) in enumerate(kf.split(vec_embeddings)):
    X_train = vec_embeddings[train_index]
    Y_train = truth_labels[train_index]
    
    X_test = vec_embeddings[test_index]
    Y_test = truth_labels[test_index]
    
    # expand input
    X_train = np.expand_dims(X_train, 2)
    X_test = np.expand_dims(X_test, 2)

    # reset model
    model = model_def()
    
    # apply training set
    history = model.fit(X_train, Y_train,
                        batch_size=10,
                        epochs=n_epochs, show=n_epochs%80==0, verbose=0);
                        # We pass some validation for
                        # monitoring validation loss and metrics
                        # at the end of each epoch
                        #validation_data=(x_val, y_val))
    
    # apply testing set
    test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=1)
    
    # record accuracy on testing
    test_accuracy.append(test_acc)
    
    print(f'Running Fold Number {k_index}')
    print(f'Size of Training Set: {len(train_index)}')
    print(f'Size of Testing Set: {len(test_index)}: {test_index}')
    print(f'Test accuracy: {test_acc}')
    print('------------------------')
    #model.predict(X_test[0:1])

1/1 [==============================] - 0s 873us/step - loss: 0.7444 - accuracy: 1.0000
Running Fold Number 0
Size of Training Set: 49
Size of Testing Set: 1: [0]
Test accuracy: 1.0
------------------------
1/1 [==============================] - 0s 682us/step - loss: 1.7427 - accuracy: 0.0000e+00
Running Fold Number 1
Size of Training Set: 49
Size of Testing Set: 1: [1]
Test accuracy: 0.0
------------------------


KeyboardInterrupt: 

In [ ]:
average_accuracy = np.mean(test_accuracy)

print(f'Average accuracy from a {n_folds}-Fold test: {average_accuracy}')